In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#visualization
import matplotlib.pyplot as plt
%matplotlib inline
import math
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

My understanding Logistic regression is a regression model where the dependent variable is categorical.
****
The prediction will be 0 or 1, Yes or No

In [ ]:
#load the dataset
data=pd.read_csv('/kaggle/input/datasets_11657_16098_train.csv')

In [ ]:
#check the shape of this dataset
data.shape

In [ ]:
data

The titanic train dataset consists of 891 rows and 12 columns.

Description of the dataset

* PassengerId - Unique Id of each passenger on the ship
* Survived - '0' for not survived & '1' for survived
* Pclass - Passenger class: '1' for 1st class, '2' for 2nd class & '3' for 3rd class
* Name - Passenger name
* Sex - gender: 'male' or 'female'
* Age - Passenger age
* SibSp - No. of siblings or spouses aborded Titanic together with the passenger
* Parch - No. of parents or children aborded Titanic together with the passenger
* Ticket - Passenger ticket number
* Fare - Passenger ticket fare
* Cabin - Passenger cabin number
* Embarked - Encoded name of city passenger embarked

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.describe(include='all')

check the null value in this dataset

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In this dataset age columns has 177 null value,Cabin columns has 687 null values and Embarked has
2 null value.

In [ ]:
#cleaning the null value
#As we can see, there are many null values under 'Cabin' column.
#687 out of 891 data points is a really high amount.and how to replace nan value with nalvale
# so i droped the Cabin column
data.drop(['Cabin'],axis=1,inplace=True)


In [ ]:
data

In [ ]:
#clean the age column 
data['Age'].fillna(data['Age'].mean(), inplace=True)

In [ ]:
data

In [ ]:
#clean the Embarked column
#replace the nan value with the top repeated element 'S'
data.Embarked.fillna(value="S",inplace=True)

In [ ]:
data

After cleaning check the null values

In [ ]:
data.isnull().sum()

# Analyzing the Dataset

In [ ]:
#drop the unrelevent column 
data.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [ ]:
#countplot for Survived column
sns.countplot(data.Survived)
plt.show()

* 0-->Not Survived
* 1-->Survived
* We can show that only 300 passengers are survived in this train.

In [ ]:
#Pclass has continous value so i can plot distplt
sns.distplot(data.Pclass[data.Survived==0])
sns.distplot(data.Pclass[data.Survived==1])
plt.legend(['Not Survived','Survived'])
plt.show()

In [ ]:
data['Pclass'].unique()

In [ ]:
sns.boxplot(x=data.Pclass,y=data.Age,data=data)
plt.show()

we can see older passenger are mostly traveling in 1st class,then 2nd and then 3rd class.

In [ ]:
sns.countplot(data.Sex,hue=data.Survived)
plt.show()

In [ ]:
sns.swarmplot(x=data.Sex,y=data.Fare,hue=data.Survived)
plt.show()


We can see that most of the female passengers are Survived in this train

In [ ]:
data['SibSp'].unique()

In [ ]:
sns.countplot(data.SibSp,hue=data.Survived)
plt.show()

In [ ]:
#not effect in our dataset so i can drop it
data.drop(['SibSp'],axis=1,inplace=True)

In [ ]:
sns.countplot(data.Parch,hue=data.Survived)
plt.show()

Above the 2 graphs, we can conclude most of the passengers on titanic are single passengers and 
the 2nd most are likely couples.

In [ ]:
sns.distplot(data.Fare[data.Survived==0])
sns.distplot(data.Fare[data.Survived==1])
plt.legend(['Not Survived','Survived'])
plt.show()

In [ ]:
data["Fare"].plot.hist(figsize=(10,10),rwidth=0.97)
plt.show()

So we can see most of the passengers on Titanic are 3rd class passengers.

In [ ]:
sns.countplot(data.Embarked,hue=data.Survived)
plt.show()

We can see that most of the passsenger are Not Survived in S.

# Pearson's Correlation

In [ ]:
corr=data.corr()
plt.figure(figsize=(12,5))
sns.heatmap(corr,annot=True,cmap='coolwarm')
plt.show()

# LOGISTIC REGRESSION

* Taitanic dataset  contain text or categorical values
*  To make the data understandable or in human readable form, the training data is often labeled in words.
* Label Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form.

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
data.Embarked=lb.fit_transform(data.Embarked)

le1=LabelEncoder()
data.Sex=le1.fit_transform(data.Sex)

In [ ]:
data

In [ ]:
#input and output selection
ip=data.drop(['Survived'],axis=1)
op=data['Survived']

In [ ]:
data

Using OheHotencoder,
We can convert 'Embarked' feature into 3 columns 'S', 'C', 'Q' with 0 for No & 1 for Yes values.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('Embarked',OneHotEncoder(),[5])],remainder='passthrough')

ip = np.array(ct.fit_transform(ip),dtype = np.str)


# Train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts=train_test_split(ip,op,test_size=0.4)

# Scalling our dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(xtr)
xtr=sc.transform(xtr)
xts=sc.transform(xts)

In [ ]:
from sklearn.linear_model import LogisticRegression
alg=LogisticRegression()

In [ ]:
#train the algorithm with the training data
alg.fit(xtr,ytr)
yp=alg.predict(xts)

# checking accuracy of the model

In [ ]:
from sklearn import metrics
cm=metrics.confusion_matrix(yts,yp)
print(cm)

In [ ]:

accuracy=metrics.accuracy_score(yts,yp)
print(accuracy)


In [ ]:
precission=metrics.precision_score(yts,yp)
print(precission)



In [ ]:

recall=metrics.recall_score(yts,yp)
print(recall)